# 02.2 - Kedro on Databricks, part 2

_Notes: This notebook is supposed to be run locally from VS Code, all with Databricks Connect_

## Kedro and Databricks Connect

**Databricks Connect** is a client library that allows you to run Spark code locally on your machine while connecting to a remote Databricks cluster for computation. It essentially lets you develop and execute Spark applications from your local IDE or notebook environment, but the actual processing happens on the Databricks cluster.

The **Databricks extension for Visual Studio Code** has several interesting features for connecting to Databricks from VS Code and perform actions sach us deploying and running Databricks Asset Bundles, manage clusters, and easily set up **Databricks Connect**.

Therefore, the two are the perfect companion for developing Kedro projects on VS Code, since you can develop on your IDE while using Databricks compute.

Follow the official documentation to

1. [Install the Databricks extension for VS Code](https://learn.microsoft.com/en-us/azure/databricks/dev-tools/vscode-ext/install)
2. [Configure the appropriate cluster](https://learn.microsoft.com/en-us/azure/databricks/dev-tools/vscode-ext/configure)
3. [Install Databricks Connect](https://learn.microsoft.com/en-us/azure/databricks/dev-tools/vscode-ext/databricks-connect)

_Note: `databricks-connect` provides its own `pyspark` top-level module, and [pip doesn't check for conflicting packages](https://github.com/pypa/pip/issues/4625), so make sure you don't have a [conflicting `pyspark` installation](https://learn.microsoft.com/en-us/azure/databricks/dev-tools/databricks-connect/python/troubleshooting#conflicting-pyspark-installations)!_

### Install the needed requirements again

In [ ]:
%pip install ../../rocketfuel
# or if the above doesn't work: %pip install -r ../../../requirements.in
%pip install hdfs s3fs

Uninstall dependencies that conflict:

In [ ]:
%pip uninstall -y kedro-mlflow

Load the Kedro ipython extension.

In [ ]:
%load_ext kedro.ipython

/Ensure your project now contains the databricks configuration created in the previous notebook. The content should be as follows:

```yaml
_uc_catalog: #your catalog location
_uc_schema: #your schema location

companies_raw:
  type: spark.SparkDataset
  filepath: /Volumes/${_uc_catalog}/${_uc_schema}/bronze/companies.csv
  file_format: csv
  load_args:
    header: True
    inferSchema: True

reviews_raw:
  type: spark.SparkDataset
  filepath: /Volumes/${_uc_catalog}/${_uc_schema}/bronze/reviews.csv
  file_format: csv
  load_args:
    header: True
    inferSchema: True

shuttles_raw:
  type: pandas.ExcelDataset
  filepath: /data/01_raw/shuttles.xlsx #There are issues loading this from the workspace, so we use a local file.
  load_args:
    engine: openpyxl

companies:
  type: databricks.ManagedTableDataset
  catalog: ${_uc_catalog}
  database: ${_uc_schema}
  table: companies
  write_mode: overwrite

reviews:
  type: databricks.ManagedTableDataset
  catalog: ${_uc_catalog}
  database: ${_uc_schema}
  table: reviews
  write_mode: overwrite

shuttles:
  type: databricks.ManagedTableDataset
  catalog: ${_uc_catalog}
  database: ${_uc_schema}
  table: shuttles
  write_mode: overwrite
```

In [ ]:
%reload_kedro ../../rocketfuel --env databricks

In [ ]:
catalog.list()

Notice how data is loaded as a PySpark DataFrame, directly from Databricks Unity Catalog!

In [ ]:
catalog._get_dataset("companies")

In [ ]:
display(catalog.load("companies"))

### Exercise 1

Codify the logic of the dummy `load_data` pipeline inside the project and sync it to the Databricks instance through `databricks-connect`. For that:

- Create the pipeline in your local development environment.
- Sync the changes to Databricks. This should happen automatically when you save a file.
- Run ```%reload_kedro``` to reload the Kedro project in Databricks.
- Try to execute the `load_data` pipeline from the Databricks notebook.
- Iterate until it works.

!Note: The existing pipelines `data_processing` and `data_science` assume they're run locally and don't reference the new dataset definitions yet. You'll need to update them to use the datasets in the Unity Catalog instead of the local references.

In [ ]:
%reload_kedro ../../rocketfuel --env databricks

In [ ]:
# The below command let's you execute the `load_data` pipeline
session.run("load_data")

## Namespaces for pipeline grouping

Namespaces allow you to group nodes, ensuring clear dependencies and separation within a pipeline while maintaining a consistent structure. Like with pipelines or tags, you can enable selective execution using namespaces, and you cannot run more than one namespace simultaneously — Kedro allows executing one namespace at a time.

Defining namespace at Pipeline-level: When applying a namespace at the pipeline level, Kedro automatically renames all inputs, outputs, and parameters within that pipeline. You will need to update your catalog accordingly. If you don’t want to change the names of your inputs, outputs, or parameters with the `namespace_name`. prefix while using a namespace, you should list these objects inside the corresponding parameters of the `pipeline()` creation function. For example:

```python
return pipeline(
    base_pipeline,
    namespace = "new_namespaced_pipeline", # With that namespace, "new_namespaced_pipeline" prefix will be added to inputs, outputs, params, and node names
    inputs={"the_original_input_name"}, # Inputs remain the same, without namespace prefix
)
```

Namespaces allow you to group your nodes and pipelines more efficiently in deployment, for example, when running your pipeline as a Databricks Job or Asset Bundle. 

To add a namespace to a Kedro pipeline, you can use the `namespace` argument in the `pipeline()` function. This argument accepts a string that will be used as a prefix for all nodes, inputs, outputs, and parameters within that pipeline. Note that you'll have to update your catalog accordingly, as Kedro expects that all inputs, outputs, and parameters within that pipeline include the namespace prefix. 

For example, to add a namespace to our `data_science` pipeline you'll have to modifify the code to:

In [ ]:
# data_science/pipeline.py

def create_pipeline(**kwargs) -> Pipeline:
    return pipeline(
        [
            node(
                func=split_data,
                inputs=["model_input_table@pandas", "params:model_options"],
                outputs=["X_train", "X_test", "y_train", "y_test"],
                name="split_data_node",
            ),
            node(
                func=train_model,
                inputs=["X_train", "y_train"],
                outputs="regressor",
                name="train_model_node",
            ),
            node(
                func=evaluate_model,
                inputs=["regressor", "X_test", "y_test"],
                outputs=None,
                name="evaluate_model_node",
            ),
        ], namespace="ds"
    )


### Update catalog.yml to add namespace prefixes to relevant datasets

```yaml
ds.model_input_table@pandas:
  type: pandas.ParquetDataset
  filepath: data/03_primary/model_input_table.parquet
```

### Update parameters.yml to add namespace prefixes to relevant parameters

```yaml
ds.model_options:
  test_size: 0.2
  random_state: 3
  features:
    - engines
    - passenger_capacity
    - crew
    - d_check_complete
    - moon_clearance_complete
    - iata_approved
    - company_rating
    - review_scores_rating
```

## Integration with Databricks MLflow

### Log Kedro runs as MLflow experiments

There are 2 types of MLflow experiments in Databricks:
- **Workspace** experiments are not associated with any notebook, and any notebook can log a run to these experiments by using the experiment ID or the experiment name. _They cannot be created inside Git folders._
- **Notebook** experiments are associated with a specific notebook. _They are note checked into source control_.

Therefore, for personal experimentation **notebook** experiments are more appropriate, and for collaboration **workspace** experiments can be created in a regular workspace folder outside of Git.

In [ ]:
%%sh
uv pip install ./rocketfuel

In [ ]:
%load_ext kedro.ipython

Kedro runs can be logged in MLflow using the [`kedro-mlflow`](https://kedro-mlflow.readthedocs.io/) community plugin.

In [ ]:
%%sh
uv pip install kedro-mlflow

`kedro-mlflow` can take [configuration](https://kedro-mlflow.readthedocs.io/en/0.14.4/source/03_experiment_tracking/01_experiment_tracking/01_configuration.html) from `conf/<environment>/mlflow.yml`, which can be used to configure the experiment name:

- For **notebook** experiments, you have to configure the experiment name to match the full path of the notebook.
- For **workspace** experiments, the experiment name would be the full path to the experiments folder in the workspace.

To this end, let's add some OmegaConf syntax to `mlflow.yml` so that the experiment name can be specified from the outside, while taking a default value if not present:

In [ ]:
%%writefile rocketfuel/conf/databricks/mlflow.yml
tracking:
  experiment:
    name: ${runtime_params:mlflow_experiment_name, ${kedro_root:}}

Let's try to set up a **notebook** experiment. For this, extract the notebook path:

In [ ]:
notebook_path = dbutils.entry_point.getDbutils().notebook().getContext().notebookPath().get()
notebook_path

And pass that as a runtime parameter to specify the experiment name:

_Note: Extra params cannot contain spaces when passed to `%reload_kedro`, see [this issue](https://github.com/kedro-org/kedro/issues/4813)_

In [ ]:
%reload_kedro rocketfuel --env databricks --params mlflow_experiment_name=$notebook_path

Now, every time a Kedro pipeline is run, it's logged as al MLflow run:

In [ ]:
session.run("load_data")

![MLflow run corresponding to a Kedro run on Databricks](./kedro-databricks-mlflow-run.png)

### Log artifacts as MLflow models in the Unity Catalog

In [ ]:
%sql
SHOW CATALOGS;

In [ ]:
# Test code
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier

import mlflow

# Train a sklearn model on the iris dataset
X, y = datasets.load_iris(return_X_y=True, as_frame=True)
clf = RandomForestClassifier(max_depth=7)
clf.fit(X, y)

# Note that the UC model name follows the pattern
# <catalog_name>.<schema_name>.<model_name>, corresponding to
# the catalog, schema, and registered model name
# in Unity Catalog under which to create the version
# The registered model will be created if it doesn't already exist
autolog_run = mlflow.last_active_run()
model_uri = "runs:/{}/model".format(autolog_run.info.run_id)  # NOTE: Can this be automatic?
mlflow.register_model(model_uri, "aza-databricks-b9b7aae-catalog.rocketfuel.iris_model")  # NOTE: Can this be automatic?

In [ ]:
session.run("__default__")

### Register models using the Databricks Unity Catalog

_**Note**: fsspec uses the DBFS API, which is not compatible with Unity Catalog according to https://github.com/fsspec/filesystem_spec/issues/1656_